Include all the packages needed for this exercise as reported below

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import math 
import statsmodels.api as sm
import sklearn
from sklearn.neighbors import KernelDensity
from scipy.integrate import nquad
import uproot
import pandas

Define two classes where one defines the caracteristics of a sample (name and sample identification), the other defines which setup to be fitted (center of the ellipse, radius or both) 

In [ ]:
class Sample:
    Signal_NonRes = -125
    TT = 1
        
class FitSetup:
    x0y0 = 1
    ab = 2
    x0y0ab = x0y0 + ab

In the following parts open the files, called "anaTuples", where already a selection is applied (Central energy scale). Create a DataFrame for the tree and calculate the total weight.

Specify for the two samples we want to analyze, Signal SM and TT and create a map for the names.

In [ ]:
#path = "/gpfs/ddn/cms/user/cmsdas/2019/hh_bbtautau/anaTuples/" #in Pisa
#path = "/eos/user/m/mgrippo/CMSDAS_2019_hh_bbtautau/anaTuples/" #on Swan

channel = #choose the channel with which you want to work

#open the file with uproot, create a pandas DataFrame (as in Significance Estimation exercise) and create total weight

#define a vector calles "samples" with sample_id and a map with names
    

Make the plot for both Signal and TT of mass of Higgs into bb candidate and of Higgs into tau pair candidate, in order to view the different behaviour of the signal and of the background. What can you notice?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,10))

n = 0
bins_x = np.linspace(0, 400, 20)
bins_y = np.linspace(0, 400, 20)
for sample in samples:
    #create an hist2d with pyplot, specifying the sample_id for each variable, 
    #set axis labels and title of the histogram
    
    n += 1
#plt.subplots_adjust(bottom=0.1, right=1.5, top=1.5)
plt.show()

Write the code for the minimization of the ellipse cut around mbb and mtautau. It has to be first calculated with 2 paramenters (the center of the ellipse or the radius of the ellipse), then also there is the possiblity to have a fit of 4 parameters

In [ ]:
class EllipseMinimization( ROOT.TPyMultiGenFunction ):
    def __init__( self, samples, x0, y0, a, b, fit_setup):
        #define the constructor of the class
        ROOT.TPyMultiGenFunction.__init__( self, self )

    def NDim(self):
        if self.fit_setup == FitSetup.x0y0ab:
            return 4
        return 2

    def DoEval(self, args):
        n = 0
        #define the elements and the args according to which setup should be run  (center or radius)
        
        #calculate the number of signal and bkg events
        
        #calculate the significance as in the previous exercise to obtain the minimum   
    
        return #significance
        
    def CalcNumEvents(self, sample_name, x0, y0, a, b):
        #define the number of events inside the ellipse reweighted with total weight

In [ ]:
def FindParams(samples, x0, y0, a, b, fit_setup):
    #use as minimizer Minuit
    ellipse = EllipseMinimization(samples, x0, y0, a, b, fit_setup)

    minimizer.SetFunction(ellipse)

    # Set the variables to be minimized!
    c_factor = 0.1
    r_factor = 5.0
    n = 0
    #set the limited variables for the different setups
        
    minimizer.Minimize()
    #result = [x for ix in minimizer.X()]
    result = [minimizer.X()[i] for i in range(ellipse.NDim())]
    return tuple(result)

Now you have to choose a starting point for the minimization. For the center values the indices of the maximum values for the mass candidate and their bin extremis should be taken

In [ ]:
sgn_id = Samples.Signal_NonRes
#create two histograms for m_bb and m_tautau (with the correct sample_id) and 
#take the maximum value and then the center considering 
#the bin extremis
print(x0, y0)

For the radius initial values should be used the 25th and 75th percentiles.

In [ ]:
#calculate the radius considering the percentile interval and the initial values calculated for the centers
print(a, b)

Calculate new center parameters starting from the initial values calculated for radius and center

In [ ]:
#use function FindParams with correct setup
print(new_x0, new_y0)

Calculate new radius parameters starting from the starting point for radius and the calculated one for centers

In [ ]:
#use FindParams with correct setup
print(new_a, new_b)

Calculate the final parameters fitting the 4 parameters, using as initial values the center and radius values obtained by previus fit in 2 dimension

In [ ]:
#use FindParams in 4 dimension
print(new_x0, new_y0, new_a, new_b)
print(final_x0, final_y0, final_a, final_b)